<a href="https://colab.research.google.com/github/jbrown544/interlingual-topic-modeling/blob/main/Interlingual_Topic_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unsupervised Thematic Discovery in Non-English Natural Language Corpora

In [17]:
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

In [18]:
# make package management current then install required versions of packages
!pip install -q -U pip setuptools wheel python-Levenshtein \
                   gensim==4.1.0 spacy==3.1.2 pyLDAvis==3.3.1

In [19]:
# gensim package information
!pip show gensim

Name: gensim
Version: 4.1.0
Summary: Python framework for fast Vector Space Modelling
Home-page: http://radimrehurek.com/gensim
Author: Radim Rehurek
Author-email: me@radimrehurek.com
License: LGPL-2.1-only
Location: /usr/local/lib/python3.7/dist-packages
Requires: smart-open, scipy, numpy
Required-by: pyLDAvis


In [20]:
# spaCy package information
!pip show spacy

Name: spacy
Version: 3.1.2
Summary: Industrial-strength Natural Language Processing (NLP) in Python
Home-page: https://spacy.io
Author: Explosion
Author-email: contact@explosion.ai
License: MIT
Location: /usr/local/lib/python3.7/dist-packages
Requires: cymem, spacy-legacy, numpy, packaging, wasabi, preshed, setuptools, srsly, requests, tqdm, thinc, typer, jinja2, blis, catalogue, typing-extensions, murmurhash, pathy, pydantic
Required-by: fastai, en-core-web-sm


In [21]:
# pyLDAvis package information
!pip show pyLDAvis

Name: pyLDAvis
Version: 3.3.1
Summary: Interactive topic model visualization. Port of the R package.
Home-page: https://github.com/bmabey/pyLDAvis
Author: Ben Mabey
Author-email: ben@benmabey.com
License: MIT
Location: /usr/local/lib/python3.7/dist-packages
Requires: scipy, pandas, numexpr, future, numpy, scikit-learn, funcy, jinja2, sklearn, joblib, gensim, setuptools
Required-by: 


In [22]:
# import dependencies

from urllib.parse import urljoin

from smart_open import open

from gensim import corpora
from gensim import models
from gensim.utils import simple_preprocess

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [23]:
# samples of french language text

french_book_url = 'https://raw.githubusercontent.com/jbrown544/interlingual-topic-modeling/main/text/books/fr/'

jules_verne_french_books = [
  urljoin(french_book_url, 'Around_the_World_in_Eighty_Days.txt'),
  urljoin(french_book_url, 'Five_Weeks_in_a_Balloon.txt'),
  urljoin(french_book_url, 'From_the_Earth_to_the_Moon.txt'),
  urljoin(french_book_url, 'Robur_the_Conqueror.txt'),
  urljoin(french_book_url, 'The_Mutineers_of_the_Bounty.txt')
]

In [24]:
# prototype of preprocessing pipeline
def preprocess_pipeline(text, 
                        deaccent=False, 
                        min_token_len=2, 
                        max_token_length=15):
  
  return simple_preprocess(text, deacc=deaccent, 
                           min_len=min_token_len, 
                           max_len=max_token_length)

In [25]:
# read and process texts into token lists
jules_verne_texts = \
  [preprocess_pipeline(open(book, encoding='latin-1').read()) \
    for book in jules_verne_french_books]

In [26]:
# construct dictionary from token lists
jules_verne_dictionary = corpora.Dictionary(jules_verne_texts)

print(jules_verne_dictionary)

Dictionary(20937 unique tokens: ['abaissait', 'abaissant', 'abaissement', 'abaissé', 'abandonna']...)


In [27]:
# create a corpus by converting token lists into a documet-term matrix
jules_verne_corpus = [jules_verne_dictionary.doc2bow(text) for text in jules_verne_texts]

In [28]:
# construct an HDP model (topics determined during processing)
jules_verne_hdp_model = models.HdpModel(jules_verne_corpus, 
                                        id2word=jules_verne_dictionary)

In [29]:
# enable visualization inline in notebook
pyLDAvis.enable_notebook()

In [30]:
# visualize suggested LDA model generated from HDP model
gensimvis.prepare(jules_verne_hdp_model.suggested_lda_model(),
                  jules_verne_corpus, jules_verne_dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster          Freq
topic                                                   
0     -0.000096  0.000172       1        1  3.003247e+01
1      0.000188  0.000103       2        1  2.575363e+01
2     -0.000272  0.000094       3        1  2.419744e+01
3     -0.000082 -0.000022       4        1  2.001632e+01
4      0.000339 -0.000171       5        1  6.790615e-05
...         ...       ...     ...      ...           ...
145   -0.000165  0.000033     146        1  2.436014e-47
146    0.000137 -0.000032     147        1  1.218007e-47
147    0.000261  0.000210     148        1  6.090034e-48
148   -0.000062 -0.000101     149        1  3.045017e-48
149   -0.000172  0.000296     150        1  3.045017e-48

[150 rows x 5 columns], topic_info=            Term          Freq      Total  Category  logprob  loglift
6202   poivriers  1.500000e+01  15.000000   Default  30.0000  30.0000
16124       fora  1.500000e+01  15.000000   Default  29.0000  29.0000
15880  effrayait  1.400000e+01  14.000000   Default  28.0000  28.0000
12731   ombragés  1.400000e+01  14.000000   Default  27.0000  27.0000
16779    narguer  1.400000e+01  14.000000   Default  26.0000  26.0000
...          ...           ...        ...       ...      ...      ...
20100    rossant  5.280172e-49  13.532849  Topic150  -9.6394   0.2479
19776   perreaux  5.174997e-49  12.701551  Topic150  -9.6595   0.2912
360     ambulant  5.220245e-49  13.752354  Topic150  -9.6508   0.2204
5484       nommé  5.168124e-49  12.770669  Topic150  -9.6609   0.2844
2210      dakota  5.190508e-49  14.385997  Topic150  -9.6565   0.1697

[7707 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
18033      1  0.251269    abaissent
18033      2  0.251269    abaissent
18033      3  0.251269    abaissent
18033      4  0.251269    abaissent
5          1  0.301702  abandonnait
...      ...       ...          ...
9358       4  0.226674          île
9361       1  0.251938        îlots
9361       2  0.335918        îlots
9361       3  0.251938        îlots
9361       4  0.167959        îlots

[22472 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150])